# Query Model Repository for best aml Model

In [1]:
from hops import model
from hops.model import Metric
MODEL_NAME="aml"
EVALUATION_METRIC="accuracy"

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
68,application_1598227897403_0032,pyspark,idle,Link,Link


SparkSession available as 'spark'.


In [2]:
best_model = model.get_best_model(MODEL_NAME, EVALUATION_METRIC, Metric.MAX)

In [3]:
print('Model name: ' + best_model['name'])
print('Model version: ' + str(best_model['version']))
print(best_model['metrics'])

Model name: aml
Model version: 2
{'accuracy': '0.975'}

# Create Model Serving of Exported Model

In [4]:
from hops import serving

In [5]:
# Create serving
model_path="/Models/" + best_model['name']
response = serving.create_or_update(model_path, MODEL_NAME, serving_type="TENSORFLOW", 
                                 model_version=best_model['version'])

Creating a serving for model aml ...
Serving for model aml successfully created

In [6]:
# List all available servings in the project
for s in serving.get_all():
    print(s.name)

aml

In [7]:
# Get serving status
serving.get_status(MODEL_NAME)

'Stopped'

# Start Model Serving Server

In [8]:
if serving.get_status(MODEL_NAME) == 'Stopped':
    serving.start(MODEL_NAME)

Starting serving with name: aml...
Serving with name: aml successfully started

In [9]:
import time
while serving.get_status(MODEL_NAME) != "Running":
    time.sleep(5) # Let the serving startup correctly
time.sleep(5)

# Send Prediction Requests to the Served Model using Hopsworks REST API

In [10]:
import numpy as np
TOPIC_NAME = serving.get_kafka_topic(MODEL_NAME)
NUM_FEATURES=128

In [13]:
import json
for i in range(1):
    data = {
                "signature_name": "serving_default", "instances": [np.random.rand(NUM_FEATURES).tolist()]
            }
    response = serving.make_inference_request(MODEL_NAME, data)
    print(response)

{'predictions': [[1.0]]}